# Assignment 2: Convolutional Autoencoders

Britt Schmitz  - i6235053 <br>
Tabea Heusel - i6323791

## 0 Setup

### 0.1 Imports

In [2]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

### 0.2 Loading Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
X = np.append(x_train, x_test, axis=0)
y = np.append(y_train, y_test, axis=0)

## 1 Autoencoder Network Implementation

### 1.1 Train, Validation, and Test 

In [4]:
# normalize
X = X / 255

# create train, validation, and test splits
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=3)
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=3)

### 1.2 Autoencoder Network

In [5]:
def create_cae():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 3, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(16, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "same", activation="relu"))
    
    return model

In [ ]:
network = create_cae()
network.compile(optimizer="sgd", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = network.fit(X_train, X_train, epochs=50, validation_data=(X_val, X_val))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.title('Evolution of Error During Training')
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
results = network.evaluate(X_test, X_test)
print(f"Test accuracy: {results[1]} \nTest error : {results[0]}")

In [ ]:
predictions = network.predict(X_test)

plt.imshow(X_test[0])
plt.show()
plt.imshow(predictions[0])
plt.show()

## 2 Latent Space Representations

### 2.1 Latent Space Size of Previous Network

In [ ]:
def compute_latent_space_size(w, k, p, s, c):
    num = w - k + 2 * p
    frac = num / s
    square = (frac + 1) ** 2
    result = square * c
    
    return result

In [ ]:
w = 8
k = 3
p = 1
s = 1
c = 16

print(compute_latent_space_size(w, k, p, s, c))

### 2.2 Correlation Between Latent Space Representation and Error

In [ ]:
def create_cae_fewer_layers():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 3, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(16, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "same", activation="relu"))
    
    return model

In [ ]:
def create_cae_more_channels():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(16, 3, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(24, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(32, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(24, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "same", activation="relu"))
    
    return model

In [ ]:
def create_cae_fewer_channels():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(4, 3, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(6, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(8, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(6, 3, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "same", activation="relu"))
    
    return model

In [ ]:
def create_cae_bigger_filter():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 5, 1, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(12, 5, 1, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(16, 5, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(12, 5, 1, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 5, 1, "same", activation="relu"))
    
    return model

In [ ]:
def create_cae_bigger_stride():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 3, 2, "same", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(12, 3, 2, "same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="same"))
    model.add(tf.keras.layers.Conv2D(16, 3, 2, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(12, 3, 2, "same", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(2))
    model.add(tf.keras.layers.Conv2D(3, 3, 2, "same", activation="relu"))
    
    return model

In [ ]:
def create_cae_valid_padding():
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv2D(8, 3, 1, "valid", activation="relu", input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPool2D(2, padding="valid"))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "valid", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(2, padding="valid"))
    model.add(tf.keras.layers.Conv2D(16, 3, 1, "valid", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(3))
    model.add(tf.keras.layers.Conv2D(12, 3, 1, "valid", activation="relu"))
    model.add(tf.keras.layers.UpSampling2D(4))
    model.add(tf.keras.layers.Conv2D(3, 3, 1, "valid", activation="relu"))
    
    return model

In [ ]:
def experiment(model, model_name):
    print(f"Experimenting with {model_name} model: \n")
    model.compile(optimizer="sgd", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
    history = model.fit(X_train, X_train, epochs=1, validation_data=(X_val, X_val))
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('Epochs')
    plt.ylabel('Binary Crossentropy')
    plt.title('Evolution of Error During Training')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    results = network.evaluate(X_test, X_test)
    print(f"Test accuracy: {results[1]} \nTest error : {results[0]}")
    
    predictions = network.predict(X_test)
    plt.imshow(X_test[0])
    plt.show()
    plt.imshow(predictions[0])
    plt.show()

In [ ]:
models = [create_cae_fewer_layers(), 
          create_cae_more_channels(),
          create_cae_fewer_channels(),
          create_cae_bigger_filter(),
          create_cae_bigger_stride(),
#          create_cae_valid_padding()
          ]

model_names = ["fewer layers",
               "more channels",
               "fewer channels",
               "bigger filter",
               "bigger stride",
#               "different padding"
               ]

for i in range(0, len(models)):
    experiment(models[i], model_names[i])

In [ ]:
print(f"Latent space sizes: \n"
      f"- Fewer Layer: {compute_latent_space_size(16, 3, 1, 1, 16)} \n"
      f"- More Channels: {compute_latent_space_size(8, 3, 1, 1, 32)} \n"
      f"- Fewer Channels: {compute_latent_space_size(8, 3, 1, 1, 8)} \n"
      f"- Bigger Filter: {compute_latent_space_size(8, 5, 1, 1, 16)} \n"
      f"- Bigger Stride: {compute_latent_space_size(2, 3, 1, 2, 16)} \n")